In [1]:
import json
import logging
import sys
import numpy as np
import pandas as pd
import greengrasssdk
# Create random name
import random
import string

In [2]:
MAX_NUM_CARPORT = 100
PUBLIC_CARPORT = 80
# init as free
carport_array = np.zeros(MAX_NUM_CARPORT, dtype=np.int8)
next_free_array = (i+1 for i in range(MAX_NUM_CARPORT))
# next_private_free = 0
next_public_free = 0
num_occupied = 0

dataset = pd.DataFrame(
    columns=["vehicle_id", "user_id", "user_name", "veh_type", "veh_state", "registered_carport_id"])
# "veh_type": 1 registered, 2 temporary(guest)
# "veh_state": >=0: occupied carport id,
user_set = {}

In [3]:
def generate_user(id):
    if id not in user_set.keys():
        user_set[id] = ''.join(
            [random.choice(string.ascii_letters + string.digits) for n in range(4)])


def generate_vehicle(id):
    veh_dic = {}
    veh_dic["vehicle_id"] = ''.join(
        [random.choice(string.ascii_letters + string.digits) for n in range(5)])
    if id not in user_set.keys():
        generate_user(id)
    veh_dic["user_id"] = id
    veh_dic["user_name"] = user_set[id]
    veh_dic["veh_type"] = 1
    veh_dic["veh_state"] = -1
    veh_dic["registered_carport_id"] = random.choice(range(PUBLIC_CARPORT))
    return veh_dic


In [4]:
def assign_carport(veh_type, veh_id):
    global num_occupied, carport_array, next_free_array, next_public_free
    assign_id = 0
    if num_occupied >= MAX_NUM_CARPORT:
        return -1
    if veh_type == 1:  # has registered carport
        filtered_df=dataset[dataset.vehicle_id == veh_id]
        # print(filtered_df)
        veh_dic={}
        veh_dic=filtered_df.to_dict(orient="list")
#         print(veh_dic)
        assign_id_list = veh_dic["registered_carport_id"]
        for assign_id in assign_id_list:
            if carport_array[assign_id] == 0:
                num_occupied += 1
                carport_array[assign_id]=1
                dataset.loc[dataset.vehicle_id == veh_id,"veh_state"] = assign_id
                return assign_id
    # not get the registered carport or guest car
    assign_id = next_public_free
    carport_array[next_public_free] = veh_type  # assign to a guest car
    next_public_free = next_free_array[assign_id]
    num_occupied += 1
    dataset.loc[dataset.vehicle_id == veh_id,"veh_state"] = assign_id
    return assign_id


def unassign_carport(carport_id):
    global num_occupied, carport_array, next_free_array, next_public_free
    if carport_array[carport_id] == 0:
        return -1
    if carport_id >= PUBLIC_CARPORT:
        next_free_array[carport_id] = next_public_free
        next_public_free = carport_id
    carport_array[carport_id] =0
    dataset.loc[dataset.veh_state == carport_id,"veh_state"]=-1
    num_occupied -= 1
    return 0

In [5]:
# init data(10 vehicles)
dataset.drop(dataset.index, inplace=True)
for i in range(10):
    dic = generate_vehicle(i)
#     print(dic)
    dataset = dataset.append(dic, ignore_index=True)
    assign_carport(1, dic["vehicle_id"])
#     print(dataset.head())

In [6]:
dataset.head(10)

,vehicle_id,user_id,user_name,veh_type,veh_state,registered_carport_id
0,bMpLA,0,SHBT,1,30,30
1,zsMI9,1,7WVw,1,61,61
2,q75CT,2,wvNX,1,66,66
3,KxbGS,3,Xe3y,1,7,7
4,D9ggG,4,v4Xj,1,72,72
5,jzcVE,5,Kvsb,1,17,17
6,ZVt0K,6,jleS,1,70,70
7,hTTQY,7,m0eh,1,3,3
8,5OuJf,8,c0Nw,1,78,78
9,tSl1g,9,HAZ9,1,36,36


In [7]:
unassign_carport(30)

0

In [8]:
dataset.head(10)

,vehicle_id,user_id,user_name,veh_type,veh_state,registered_carport_id
0,bMpLA,0,SHBT,1,-1,30
1,zsMI9,1,7WVw,1,61,61
2,q75CT,2,wvNX,1,66,66
3,KxbGS,3,Xe3y,1,7,7
4,D9ggG,4,v4Xj,1,72,72
5,jzcVE,5,Kvsb,1,17,17
6,ZVt0K,6,jleS,1,70,70
7,hTTQY,7,m0eh,1,3,3
8,5OuJf,8,c0Nw,1,78,78
9,tSl1g,9,HAZ9,1,36,36


In [11]:
print(carport_array[66])

1


In [ ]:
assign_carport(1, dic["vehicle_id"])